In [1]:
!pip install tslearn

     |████████████████████████████████| 793 kB 5.2 MB/s 


In [ ]:


def NNsearch():

  import random
  import math
  from matplotlib import pyplot



  import matplotlib.pyplot as plt

  from tslearn.clustering import KShape
  from tslearn.datasets import CachedDatasets
  from tslearn import metrics



    
  import time
  import random
  import math
  from matplotlib import pyplot
  import numpy as np
  from tslearn.datasets import UCR_UEA_datasets
  from tslearn.clustering import silhouette_score





  uea_ucr = UCR_UEA_datasets()

  X_train, y_train, X_test, y_test = uea_ucr.load_dataset('FordA')
  X_train = np.concatenate([X_train, X_test])
  result=[]
  tmp_result=[]
  query=[]
  for i in range(len(X_train)):
    tmp_result=[]
    a = X_train[i].tolist()
    for j in range(len(X_train[0])):
      tmp_result.append(a[j][0])
    result.append(tmp_result) 
  print(len(result))
  print(len(result[0]))
  time_series_data_length=len(result[0])
  data_num=len(result)-1 #データベースのデータの数
  #ランダムにクエリを選ぶ
  query_num = random.randint(0, data_num)
  query=result[query_num]
  result.pop(query_num)


  x_axis = [i for i in range(0,time_series_data_length)]
  # for i in range(len(result)):  
  #   pyplot.plot(x_axis, result[i], label=str(i), color="r")
  # pyplot.show()  

  def cal_paa_onedata(database_list, dimension_num):
    len_list = len(database_list)
    c =[]
    width = len_list//(dimension_num-1)
    for i in range(0,dimension_num-1):
      tmp_c=0
      for j in range(i*width,(i+1)*width):
        tmp_c = tmp_c + database_list[j]
      c.append(tmp_c/width)
    tmp_c=0
    last_count=0  
    for i in range(width*(dimension_num-1), len_list):
      last_count=last_count+1
      tmp_c = tmp_c + database_list[i]
    c.append(tmp_c/(last_count))
    return c

  def cal_paa_manydata(database_lists, dimension_num):
    results = []
    for database_list in database_lists:
      results.append(cal_paa_onedata(database_list, dimension_num))
    return results  

  #クラスタリング（k-shape）
  def clustering(databases, n_clus):
    seed = 0
    #pythonのlistをnumpyのarrayに
    databases = np.array(databases)
    #kShape clusteringしている
    ks = KShape(n_clus, verbose=True, random_state=seed)
    # 配列clusにdatabasesのcluteringの結果を入れる
    clus = ks.fit_predict(databases)
    clus = clus.tolist()
    centers = ks.cluster_centers_
    return clus, ks, centers

  
  query_paa=[] #クエリのpaa
  database_paa=[] #databaseのpaa
  database_clus=[] #databaseのクラス
  clus_center = [] #クラスタ中心（正規化している）
  paa_dimension_num =  100 #次元削減後の次元
  database_num=data_num
  n_clus = -1 #データベースのクラス数



  """
  PAAする
  """
  query_paa = cal_paa_onedata(query,paa_dimension_num)
  database_paa = cal_paa_manydata(result,paa_dimension_num)

  """
  グルーピング
  """
  max_sil=-100000000000000000000000000000000000000000000

  small_database_paa = random.sample(database_paa, 100)

  for i in range(3,10):
    tmp_database_clus, model, tmp_centers = clustering(small_database_paa, i)
    dtw_sil = silhouette_score(small_database_paa, tmp_database_clus, metric="dtw")
    if max_sil<dtw_sil:
      max_sil=dtw_sil
      n_clus=i
  print(n_clus)
  database_clus, model, centers = clustering(database_paa, n_clus)

  for i in range(len(centers)):
    raveled_center = np.ravel(centers[i])
    clus_center.append(raveled_center.tolist())
  #print(clus_center)  

  #knnグラフ作る
  search_num=n_clus//3

  big=99999999999999999999999999999999999999999999999999999999999999999999
  knn_dtw=[[big for i in range(n_clus)] for j in range(n_clus)]
  knn_graph=[[big for i in range(n_clus)] for j in range(n_clus)]
  near_group = [[-1 for i in range(search_num)]for j in range(n_clus)] #i番目のクラスタにj番目に近いクラスタ番号が入っている

  for i in range(n_clus):
    for j in range(n_clus):
      if i==j:
        continue
      knn_dtw[i][j] = metrics.dtw(clus_center[i], clus_center[j])
  print(knn_dtw)

  for i in range(n_clus):
    for j in range(search_num):
      min_num = min(knn_dtw[i])
      ind = knn_dtw[i].index(min_num)
      near_group[i][j] = ind
      knn_graph[i][ind] = j
      knn_dtw[i][ind]=big
  print(knn_graph)  
  print(near_group)

  #pytorch

  import torch
  from torch import nn
  from torch.utils.data import DataLoader
  from torchvision import datasets
  from torchvision.transforms import ToTensor, Lambda, Compose
  import matplotlib.pyplot as plt
  import random
  import numpy as np
  from sklearn.cluster import KMeans
  from torch.nn.functional import one_hot
  #batch_sizeはデータ数の約数
  def pytorch(database_paa, database_label, query_paa):
    database_label = one_hot(torch.tensor(database_label, dtype=torch.int64), num_classes = n_clus)
    database_label = database_label.float()
    #batch_sizeはデータ数の約数
    batch_size=10
    X = np.array(database_paa)
    X = torch.tensor(X, dtype=torch.float32)
    y = database_label
    Dataset = torch.utils.data.TensorDataset(X, y)
    Loader = torch.utils.data.DataLoader(dataset=Dataset, batch_size=batch_size, shuffle=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # modelを定義します
    class NeuralNetwork(nn.Module):
      def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
          nn.Linear(paa_dimension_num, 8),
          nn.ReLU(),
          nn.Dropout(0.2),
          nn.Linear(8, n_clus),
          nn.ReLU()
        )

      def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

    model = NeuralNetwork().to(device)
    print(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    def train(dataloader, model, loss_fn, optimizer):
      size = len(dataloader.dataset)
      alpha = 0.01 # 正則化パラメータ
      for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
          
          # 損失誤差を計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # パラメータのL2ノルムの二乗を損失関数に足す
        l2 = torch.tensor(0., requires_grad=True)
        for w in model.parameters():
          l2 = l2 + torch.norm(w)**2
        loss = loss + alpha*l2
          
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 2 == 0:
          loss, current = loss.item(), batch * len(X)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    epochs = 300
    train_start_time=time.time()
    for t in range(epochs):
      print(f"Epoch {t+1}\n-------------------------------")
      train(Loader, model, loss_fn, optimizer)
    train_end_time=time.time()  
    train_time=train_end_time - train_start_time
    print("Done!")

    #testフェーズ
    model.eval()
    model_start_time = time.time()
    #pytorchの出力
    new_query_paa=[]
    new_query_paa.append(query_paa)
    with torch.no_grad():
      pred = model(torch.tensor(new_query_paa, dtype=torch.float32))
      pred = torch.argmax(pred, dim=1)
      pred = pred.item()
    model_end_time = time.time()
    model_time = model_end_time - model_start_time
    return pred, model_time, train_time

  import time
  #pytorchのtrainフェーズ
  pred, model_time, train_time = pytorch(database_paa, database_clus, query_paa)
  query_group=pred

  #lb_keoghはクエリとデータベースの時系列長が同じでないといけない
  #もし時系列長が異なるものにも対応したいならこれは使わない
  #queとdataを逆にしてもlowerbouding関数として機能する
  def lb_keogh(que, data):
    w = time_series_data_length//2
    keogh=0
    upper =[]
    lower =[]
    for i in range(len(que)):
      if i<w:
        u=max(que[0:i+w])
        l=min(que[0:i+w])
        upper.append(u)
        lower.append(l)
      elif i>len(que)-w:
        u=max(que[i-w:])
        l=min(que[i-w:])
        upper.append(u)
        lower.append(l)
      else:
        u=max(que[i-w:i+w])
        l=min(que[i-w:i+w])
        upper.append(u)
        lower.append(l)

      if data[i]>u:
        keogh = keogh + (data[i]-u)**2
      if data[i]<l:
        keogh = keogh + (l-data[i])**2
    return keogh**0.5
  #探索
  tmp_dtw = 1000000000000000000000000000000000000000000000000
  ans=0
  ans_group=-1
  print(near_group[query_group])
  dtw_start_time = time.time()
  for i in range(len(database_clus)):
    if (query_group == database_clus[i]) or (database_clus[i] in near_group[query_group]):#queryグループかそのグループに近いグループのデータのみ探索
      sim = metrics.dtw(query, result[i], global_constraint="sakoe_chiba", sakoe_chiba_radius=time_series_data_length//2)
      if sim < tmp_dtw:
        tmp_dtw = sim
        ans = i
        ans_group = database_clus[i]

  dtw_end_time = time.time()
  dtw_time=dtw_end_time-dtw_start_time      
  print("ans is")
  print(ans)  
  print("dtwの値は")
  print(tmp_dtw)
  print("モデル生成でかかった時間は")
  print(model_time)   
  print("dtw計算でかかった時間は")
  print(dtw_time) 
  print("合計かかった時間は")
  print(model_time+dtw_time)
  print("結果のグループは")
  print(ans_group)

  sug1_ans = ans
  sug1_time = model_time+dtw_time
  sug_dtw=tmp_dtw

  pyplot.plot(x_axis, result[ans],  color="r")
  pyplot.plot(x_axis, query, color="b")
  pyplot.show()  

  #lbkeogh
  from tslearn import metrics
  import time

  tmp_dtw = 1000000000000000000000000000000000000000000000000
  ans=0
  ans_group=-1
  keogh_list=[]
  print(near_group[query_group])
  dtw_start_time = time.time()
  for i in range(len(database_clus)):#lbできてないからどうにかする
    if (query_group == database_clus[i]) or (database_clus[i] in near_group[query_group]):#queryグループかそのグループに近いグループのデータのみ探索
      tmp_keogh_list=[]
      tmp_keogh_list.append(lb_keogh(query, result[i]))
      tmp_keogh_list.append(i)
      keogh_list.append(tmp_keogh_list)
  print(keogh_list[:100])
  keogh_list.sort()
  print(keogh_list[:100])

  for i in range(len(keogh_list)):
    if tmp_dtw < keogh_list[i][0]:
      break
    sim = metrics.dtw(query, result[keogh_list[i][1]], global_constraint="sakoe_chiba", sakoe_chiba_radius=time_series_data_length//2)
    if sim < tmp_dtw:
      tmp_dtw = sim
      ans = keogh_list[i][1]
      ans_group = database_clus[ans]

  dtw_end_time = time.time()
  dtw_time=dtw_end_time-dtw_start_time      
  print("ans is")
  print(ans)  
  print("dtwの値は")
  print(tmp_dtw)
  print("モデル生成でかかった時間は")
  print(model_time)   
  print("dtw計算でかかった時間は")
  print(dtw_time) 
  print("合計かかった時間は")
  print(model_time+dtw_time)
  print("結果のグループは")
  print(ans_group)

  sug2_ans=ans
  sug2_time=model_time+dtw_time


  pyplot.plot(x_axis, result[ans],  color="r")
  pyplot.plot(x_axis, query, color="b")
  pyplot.show()  





  tmp_dtw = 1000000000000000000000000000000000000000000000000
  ans=0
  rank=[]
  dtw_start_time=time.time()
  for i in range(len(database_clus)):
    tmp_rank=[]
    sim = metrics.dtw(query, result[i], global_constraint="sakoe_chiba", sakoe_chiba_radius=time_series_data_length//2)
    tmp_rank.append(sim)
    tmp_rank.append(i)
    rank.append(tmp_rank)
    if sim < tmp_dtw:
      tmp_dtw = sim
      ans = i
  dtw_end_time = time.time()   
  rank.sort()
  dtw_time = dtw_end_time-dtw_start_time 
  print("ans is")
  print(ans)  
  print("dtwの値は")  
  print(tmp_dtw)  
  print("dtwの計算にかかった時間は")
  print(dtw_time)
  print("グループは")
  print(database_clus[ans])

  ranking=-1
  for i in range(len(rank)):
    if rank[i][1]==sug1_ans:
      ranking=i
      break

  
  nor1_ans=ans
  nor1_time=dtw_time
  nor_dtw=tmp_dtw

  pyplot.plot(x_axis, result[ans],  color="r")
  pyplot.plot(x_axis, query, color="b")
  pyplot.show()  

  #baseline
  from tslearn import metrics

  tmp_dtw = 1000000000000000000000000000000000000000000000000
  ans=0
  dtw_start_time=time.time()
  for i in range(len(database_clus)):
    keogh = lb_keogh(query, result[i])
    if keogh > tmp_dtw:
      continue
    sim = metrics.dtw(query, result[i], global_constraint="sakoe_chiba", sakoe_chiba_radius=time_series_data_length//2)
    if sim < tmp_dtw:
      tmp_dtw = sim
      ans = i
  dtw_end_time = time.time()   
  dtw_time = dtw_end_time-dtw_start_time 
  print("ans is")
  print(ans)  
  print("dtwの値は")  
  print(tmp_dtw)  
  print("dtwの計算にかかった時間は")
  print(dtw_time)
  print("グループは")
  print(database_clus[ans])

  nor2_ans=ans
  nor2_time=dtw_time



  pyplot.plot(x_axis, result[ans],  color="r")
  pyplot.plot(x_axis, query, color="b")
  pyplot.show() 

  return sug1_ans, sug1_time, sug2_ans, sug2_time, nor1_ans, nor1_time, nor2_ans, nor2_time, ranking, sug_dtw, nor_dtw, query_num, model_time, train_time








In [ ]:
try_num=10
sug1_anses=[]#提案手法一つ目の答え
sug2_anses=[]#提案手法二つ目の答え
nor1_anses=[]#比較手法一つ目の答え
nor2_anses=[]#比較手法二つ目の答え
sug1_times=[]#提案手法一つ目のかかった時間
sug2_times=[]#提案手法二つ目のかかった時間
nor1_times=[]#比較手法一つ目のかかった時間
nor2_times=[]#比較手法二つ目のかかった時間
ranks=[]#提案手法が出した答えが実際は何番目に近かったか
sug_dtws=[]#提案手法が出した答えのdtw
nor_dtws=[]#比較手法が出した答えのdtw
query_nums=[]#クエリのインデックス
model_times=[]
train_times=[]
for i in range(try_num):
  sug1_ans, sug1_time, sug2_ans, sug2_time, nor1_ans, nor1_time, nor2_ans, nor2_time, ranking, sug_dtw, nor_dtw, query_num, model_time, train_time = NNsearch()
  sug1_anses.append(sug1_ans)
  sug2_anses.append(sug2_ans)
  nor1_anses.append(nor1_ans)
  nor2_anses.append(nor2_ans)
  sug1_times.append(sug1_time)
  sug2_times.append(sug2_time)
  nor1_times.append(nor1_time)
  nor2_times.append(nor2_time)
  ranks.append(ranking)
  sug_dtws.append(sug_dtw)
  nor_dtws.append(nor_dtw)
  query_nums.append(query_num)
  model_times.append(model_time)
  train_times.append(train_time)
print("おわり") 

KeyboardInterrupt: ignored

In [ ]:
print(sug1_anses)
print(nor1_anses)
print(sug1_times)
print(sug2_times)
print(nor1_times)
print(nor2_times)
print(ranks)
print(sug_dtws)
print(nor_dtws)
print(model_times)
print(train_times)

[3767, 311, 4053, 3445, 2320, 106, 3626, 1561, 2210, 3220]
[3767, 896, 4053, 3445, 2038, 106, 3626, 1561, 2210, 1134]
[20.015757083892822, 25.604652643203735, 25.898329734802246, 25.64721417427063, 26.558504819869995, 25.46126627922058, 24.79819679260254, 23.965221166610718, 23.503819465637207, 26.122637510299683]
[43.94945216178894, 53.89809727668762, 54.04737877845764, 53.00268816947937, 56.09020185470581, 57.97955942153931, 51.47565269470215, 50.92583465576172, 50.16351342201233, 53.37704944610596]
[38.7491238117218, 40.86348056793213, 38.334864377975464, 38.00333309173584, 38.38239002227783, 37.10911273956299, 37.52689003944397, 38.082640171051025, 36.726154088974, 39.64162087440491]
[82.29954028129578, 82.40213179588318, 82.0105528831482, 80.99000287055969, 79.93235802650452, 79.18457126617432, 77.13774538040161, 84.17176413536072, 80.4030659198761, 77.66748929023743]
[0, 5, 0, 0, 1, 0, 0, 0, 0, 2]
[7.627564717902365, 5.250559213132667, 6.933916689403241, 6.117858029766089, 8.2318

In [ ]:
sug1_times=[1.9314532279968262, 3.3679556846618652, 1.8911499977111816, 3.1125473976135254, 2.4158191680908203, 3.303494691848755, 3.100562572479248, 4.186818838119507, 2.430891990661621, 2.287393569946289]
sug2_times=[2.6389949321746826, 4.451390027999878, 2.6555557250976562, 3.985022783279419, 3.158916711807251, 4.318218231201172, 4.0376434326171875, 5.368358612060547, 3.0647988319396973, 3.041095733642578]
nor1_times=[4.778954744338989, 5.095137119293213, 5.138018608093262, 4.776839733123779, 4.7857749462127686, 4.866624355316162, 4.709210395812988, 4.655661582946777, 4.911618709564209, 4.672225713729858]
nor2_times=[6.698792219161987, 6.617241144180298, 6.64945387840271, 6.222443103790283, 6.188751459121704, 6.334961652755737, 6.013357400894165, 6.400184631347656, 6.386327743530273, 6.339659214019775]

print(sum(sug1_times)/len(sug1_times))
print(sum(sug2_times)/len(sug2_times))
print(sum(nor1_times)/len(nor1_times))
print(sum(nor2_times)/len(nor2_times))
print(sum(model_times)/len(model_times))
print(sum(train_times)/len(train_times))


2.802808713912964
3.6719995021820067
4.8390065908432005
6.385117244720459
0.0008515357971191406
183.25970804691315


In [ ]:
from tslearn.datasets import UCR_UEA_datasets
uea_ucr = UCR_UEA_datasets()

X_train, y_train, X_test, y_test = uea_ucr.load_dataset("FordA")
num=len(X_train)+len(X_test)
length = len(X_train[0])
print(num)
print(length)

4921
500


In [ ]:
print(sum(nor2_times)/len(nor2_times))

411.3110678434372


In [ ]:
print(sum(sug_dtws)/len(nor_dtws))

1.011742014476367


In [ ]:
print(nor_dtws[6]/sug_dtws[6])

0.9997925709907595


In [ ]:
from tslearn.datasets import UCR_UEA_datasets
import numpy as np
uea_ucr = UCR_UEA_datasets()

X_train, y_train, X_test, y_test = uea_ucr.load_dataset('StarLightCurves')
X_train = np.concatenate([X_train, X_test])
print(len(X_train))
print(len(X_train[0]))

9236
1024
